In [ ]:
!pip install scikeras

In [ ]:
# Importing Libraries
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from tensorflow.keras import losses
# Import KerasClassifier from scikeras.wrappers instead
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, precision_recall_curve, f1_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn import tree
import graphviz
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/bank_fraud.csv")

In [ ]:
df

,Transaction_id,Customer_id,Amount,Transaction_type,Longitude,Latitude,Transaction_Timestamp,Hour_of_Day,Day_of_Week,IP_Address,avg_spending,avg_transaction_count,Fraud
0,6f2086b3,d2f8e5,2813.49,Withdraw,-36.975757,-66.686750,2024-11-17 22:11:31,22,6,116.179.105.67,4166.28,43,1
1,6309e287,039458,9234.37,Payment,-25.430183,-8.244230,2024-10-26 08:56:05,8,5,28.8.98.22,737.00,10,0
2,a79540f4,92e065,3071.12,Payment,71.924224,-39.178163,2024-09-15 22:28:08,22,6,70.98.246.116,2650.13,19,1
3,f84e3803,f69985,3766.44,Transfer,-54.019763,-67.034887,2024-05-23 22:39:17,22,3,159.151.173.114,233.90,13,0
4,7972e221,fcf82f,3252.85,Withdraw,-33.344697,36.257655,2024-08-09 08:14:21,8,4,102.247.251.122,3700.50,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,8d1c31d5,a2e019,2451.36,Transfer,-25.382137,-29.043088,2025-01-16 03:07:18,3,3,197.103.241.255,2650.50,9,1
19996,94892616,a062e5,1720.55,Transfer,107.153788,-2.158004,2024-12-09 15:07:13,15,0,144.99.102.120,2761.44,8,1
19997,3f3f7c97,72d7f0,7682.03,Payment,136.699252,56.249340,2025-01-02 23:56:31,23,3,175.99.199.202,660.99,36,0
19998,4aafd148,d1b3f6,8072.62,Transfer,99.556804,40.305875,2024-12-03 01:40:40,1,1,14.245.48.26,2529.89,17,0


In [ ]:
print(df.dtypes)

Transaction_id            object
Customer_id               object
Amount                   float64
Transaction_type          object
Longitude                float64
Latitude                 float64
Transaction_Timestamp     object
Hour_of_Day                int64
Day_of_Week                int64
IP_Address                object
avg_spending             float64
avg_transaction_count      int64
Fraud                      int64
dtype: object


In [ ]:
# Initialize LabelEncoder
label_encoders = {}

# Convert all object (categorical) columns to int64 using LabelEncoder
for col in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # Convert to string to handle NaNs
    label_encoders[col] = le

print(df.dtypes)

Transaction_id             int64
Customer_id                int64
Amount                   float64
Transaction_type           int64
Longitude                float64
Latitude                 float64
Transaction_Timestamp      int64
Hour_of_Day                int64
Day_of_Week                int64
IP_Address                 int64
avg_spending             float64
avg_transaction_count      int64
Fraud                      int64
dtype: object


In [ ]:
X = df.drop(columns=['Fraud'])
y = df['Fraud']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
def CMatrix(CM, labels=['non-fraud', 'fraud']):
    df = pd.DataFrame(data=CM, index=labels, columns=labels)
    df.index.name = 'actual'
    df.columns.name = 'predict'
    df.loc['Total'] = df.sum()
    df['Total'] = df.sum(axis=1)
    return df


metrics = pd.DataFrame(index=['accuracy', 'precision', 'recall'],
                       columns=['NULL', 'LogisticReg', 'ClassTree', 'NaiveBayes', 'KNN','SVM','ANN'])

In [ ]:
y_pred_test = np.repeat(y_train.value_counts().idxmax(), y_test.size)

metrics.loc['accuracy', 'NULL'] = accuracy_score(
    y_pred=y_pred_test, y_true=y_test)
metrics.loc['precision', 'NULL'] = precision_score(
    y_pred=y_pred_test, y_true=y_test)
metrics.loc['recall', 'NULL'] = recall_score(y_pred=y_pred_test, y_true=y_test)
metrics.loc['f1_score','NULL'] = f1_score(y_pred=y_pred_test, y_true = y_test)

CM = confusion_matrix(y_pred=y_pred_test, y_true=y_test)
CMatrix(CM)

predict,non-fraud,fraud,Total
actual,,,
non-fraud,0,1823,1823
fraud,0,4177,4177
Total,0,6000,6000


In [ ]:
# A. Logistic Regression
# 1. Create an instance of the estimator
logistic_regression = LogisticRegression(n_jobs=-1, random_state=15)

# 2. Use the training data to train the estimator
logistic_regression.fit(X_train, y_train)

# 3. Evaluate the model
y_pred_test = logistic_regression.predict(X_test)

metrics.loc['accuracy', 'LogisticReg'] = accuracy_score(
    y_pred=y_pred_test, y_true=y_test)
metrics.loc['precision', 'LogisticReg'] = precision_score(
    y_pred=y_pred_test, y_true=y_test)
metrics.loc['recall', 'LogisticReg'] = recall_score(
    y_pred=y_pred_test, y_true=y_test)
metrics.loc['f1_score','LogisticReg'] = f1_score(y_pred=y_pred_test, y_true = y_test)

# Confusion matrix
CM = confusion_matrix(y_pred=y_pred_test, y_true=y_test)
CMatrix(CM)



predict,non-fraud,fraud,Total
actual,,,
non-fraud,0,1823,1823
fraud,0,4177,4177
Total,0,6000,6000


In [ ]:
# B. Classification Trees
# 1. Create an instance of the estimator
class_tree = DecisionTreeClassifier(
    min_samples_split=30, min_samples_leaf=10, random_state=10)

# 2. Use the training data to train the estimator
class_tree.fit(X_train, y_train)

# 3. Evaluate the model
y_pred_test = class_tree.predict(X_test)

metrics.loc['accuracy', 'ClassTree'] = accuracy_score(
    y_pred=y_pred_test, y_true=y_test)
metrics.loc['precision', 'ClassTree'] = precision_score(
    y_pred=y_pred_test, y_true=y_test)
metrics.loc['recall', 'ClassTree'] = recall_score(
    y_pred=y_pred_test, y_true=y_test)
metrics.loc['f1_score','ClassTree'] = f1_score(y_pred=y_pred_test, y_true = y_test)

# Confusion matrix
CM = confusion_matrix(y_pred=y_pred_test, y_true=y_test)
CMatrix(CM)

predict,non-fraud,fraud,Total
actual,,,
non-fraud,477,1346,1823
fraud,1146,3031,4177
Total,1623,4377,6000


In [ ]:
# C. Naive Bayes Classifier
# 1. Create an instance of the estimator
NBC = GaussianNB()

# 2. Use the training data to train the estimator
NBC.fit(X_train, y_train)

# 3. Evaluate the model
y_pred_test = NBC.predict(X_test)

metrics.loc['accuracy', 'NaiveBayes'] = accuracy_score(
    y_pred=y_pred_test, y_true=y_test)
metrics.loc['precision', 'NaiveBayes'] = precision_score(
    y_pred=y_pred_test, y_true=y_test)
metrics.loc['recall', 'NaiveBayes'] = recall_score(
    y_pred=y_pred_test, y_true=y_test)
metrics.loc['f1_score','NaiveBayes'] = f1_score(y_pred=y_pred_test, y_true = y_test)
# Confusion matrix
CM = confusion_matrix(y_pred=y_pred_test, y_true=y_test)
CMatrix(CM)

predict,non-fraud,fraud,Total
actual,,,
non-fraud,0,1823,1823
fraud,0,4177,4177
Total,0,6000,6000


In [ ]:
# E. - KNN
# 1. Create an instance of the estimator
KNN = KNeighborsClassifier()

# 2. Use the training data to train the estimator
KNN.fit(X_train, y_train)

# 3. Evaluate the model
y_pred_test = KNN.predict(X_test)
# metrics.loc['classification_report', 'KNN'] = classification_report(y_pred=y_pred_test, y_true=y_test)
metrics.loc['accuracy', 'KNN'] = accuracy_score(
    y_pred=y_pred_test, y_true=y_test)
metrics.loc['precision', 'KNN'] = precision_score(
    y_pred=y_pred_test, y_true=y_test)
metrics.loc['recall', 'KNN'] = recall_score(y_pred=y_pred_test, y_true=y_test)
metrics.loc['f1_score','KNN'] = f1_score(y_pred=y_pred_test, y_true = y_test)

# #Confusion matrix
CM = confusion_matrix(y_pred=y_pred_test, y_true=y_test)
CMatrix(CM)


predict,non-fraud,fraud,Total
actual,,,
non-fraud,302,1521,1823
fraud,662,3515,4177
Total,964,5036,6000


In [ ]:
#Standardize features (important for ANN & SVM)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
# Define ANN model
ann = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=200, random_state=42)

# Train ANN
ann.fit(X_train, y_train)

# Predictions
y_pred_ann = ann.predict(X_test)

# Evaluate ANN
print("🔹 ANN Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_ann):.2f}")
print(f"Precision: {precision_score(y_test, y_pred_ann):.2f}")
print(f"Recall: {recall_score(y_test, y_pred_ann):.2f}")
print(f"F1 Score: {f1_score(y_test, y_pred_ann):.2f}")

metrics.loc['accuracy', 'ANN'] = accuracy_score(
    y_pred=y_pred_ann, y_true=y_test)
metrics.loc['precision', 'ANN'] = precision_score(
    y_pred=y_pred_ann, y_true=y_test)
metrics.loc['recall', 'ANN'] = recall_score(
    y_pred=y_pred_ann, y_true=y_test)
metrics.loc['f1_score','ANN'] = f1_score(y_pred=y_pred_ann, y_true = y_test)

🔹 ANN Performance:
Accuracy: 0.59
Precision: 0.70
Recall: 0.73
F1 Score: 0.71


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
from sklearn.svm import SVC

# Define SVM model
svm = SVC(kernel='rbf', C=1.0, probability=True, random_state=42)  # You can change kernel to 'linear' or 'poly'

# Train SVM
svm.fit(X_train, y_train)

# Predictions
y_pred_svm = svm.predict(X_test)

# Evaluate SVM
print("🔹 SVM Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")
print(f"Precision: {precision_score(y_test, y_pred_svm):.2f}")
print(f"Recall: {recall_score(y_test, y_pred_svm):.2f}")
print(f"F1 Score: {f1_score(y_test, y_pred_svm):.2f}")

🔹 SVM Performance:
Accuracy: 0.70
Precision: 0.70
Recall: 1.00
F1 Score: 0.82


In [ ]:
# Scores for all the models
100*metrics

,NULL,LogisticReg,ClassTree,NaiveBayes,KNN,SVM,ANN
accuracy,69.616667,69.616667,58.466667,69.616667,63.616667,NaN,59.05
precision,69.616667,69.616667,69.248344,69.616667,69.797458,NaN,69.563239
recall,100.0,100.0,72.564041,100.0,84.151305,NaN,73.210438
f1_score,82.087059,82.087059,70.86743,82.087059,76.305221,NaN,71.340254


In [ ]:
!pip install xgboost scikit-learn pandas numpy

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

X = df.drop(columns=["Transaction_id", "Fraud"])  # Remove non-predictive columns
y = df["Fraud"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:58:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [ ]:
y_pred_rf = rf_model.predict(X_test)
y_pred_xgb = xgb_model.predict(X_test)

In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"{model_name} Performance:")
    print(f" Accuracy: {acc:.2f}")
    print(f" Precision: {precision:.2f}")
    print(f" Recall: {recall:.2f}")
    print(f" F1 Score: {f1:.2f}")
    print("-" * 40)

evaluate_model(y_test, y_pred_rf, "Random Forest")
evaluate_model(y_test, y_pred_xgb, "XGBoost")

Random Forest Performance:
 Accuracy: 0.70
 Precision: 0.70
 Recall: 0.99
 F1 Score: 0.82
----------------------------------------
XGBoost Performance:
 Accuracy: 0.70
 Precision: 0.70
 Recall: 0.99
 F1 Score: 0.82
----------------------------------------


In [ ]:
# Initialize models
svm_model = SVC(kernel='rbf', probability=True, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Train models
svm_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

# Predictions
svm_preds = svm_model.predict(X_test)
rf_preds = rf_model.predict(X_test)
xgb_preds = xgb_model.predict(X_test)

# Function to evaluate model
def evaluate_model(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f"{model_name} Performance:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print("-" * 40)

# Evaluate all models
evaluate_model(y_test, svm_preds, "SVM")
evaluate_model(y_test, rf_preds, "Random Forest")
evaluate_model(y_test, xgb_preds, "XGBoost")

In [ ]:
!pip install imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# 🔹 Split resampled data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# 🔥 Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:59:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [ ]:
y_pred_rf = rf_model.predict(X_test)
y_pred_xgb = xgb_model.predict(X_test)


In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"{model_name} Performance (After SMOTE):")
    print(f"Accuracy: {acc:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print("-" * 40)

evaluate_model(y_test, y_pred_rf, "Random Forest")
evaluate_model(y_test, y_pred_xgb, "XGBoost")

Random Forest Performance (After SMOTE):
Accuracy: 0.69
Precision: 0.69
Recall: 0.70
F1 Score: 0.70
----------------------------------------
XGBoost Performance (After SMOTE):
Accuracy: 0.63
Precision: 0.63
Recall: 0.63
F1 Score: 0.63
----------------------------------------


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
rf_params = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10],
    'min_samples_split': [2, 5],
}

rf_random =  RandomizedSearchCV(RandomForestClassifier(random_state=42), rf_params,n_iter=5, cv=3, scoring='f1', n_jobs=2,random_state=42)
rf_random.fit(X_train, y_train)

best_rf = rf_random.best_estimator_
print("✅ Best Random Forest Params:", rf_random.best_params_)

✅ Best Random Forest Params: {'n_estimators': 100, 'min_samples_split': 2, 'max_depth': 10}


In [ ]:
xgb_params = {
    'n_estimators': [100, 200],
    'learning_rate': [0.1],  # Keep only 1 value
    'max_depth': [3, 5],  # Reduce options
    'subsample': [0.8]
}

xgb_random = RandomizedSearchCV(XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
                         xgb_params, cv=3, scoring='f1', n_jobs=-1)
xgb_random.fit(X_train, y_train)

best_xgb = xgb_random.best_estimator_
print("✅ Best XGBoost Params:", xgb_random.best_params_)

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:00:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Best XGBoost Params: {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.1}


In [ ]:
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"\n{model_name} Performance (After Tuning):")
    print(f" Accuracy: {acc:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f" Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print("-" * 40)

evaluate_model(best_rf, X_test, y_test, "Random Forest")
evaluate_model(best_xgb, X_test, y_test, "XGBoost")


Random Forest Performance (After Tuning):
 Accuracy: 0.63
Precision: 0.63
 Recall: 0.64
F1 Score: 0.64
----------------------------------------

XGBoost Performance (After Tuning):
 Accuracy: 0.66
Precision: 0.66
 Recall: 0.65
F1 Score: 0.65
----------------------------------------


In [ ]:
#Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from xgboost import XGBClassifier

# Define faster models
rf = RandomForestClassifier(n_estimators=20, max_depth=3, random_state=42, n_jobs=-1)
xgb = XGBClassifier(n_estimators=20, max_depth=3, learning_rate=0.1, tree_method='hist', random_state=42)

# Create Voting Classifier (Removed SVM)
voting_clf = VotingClassifier(estimators=[('rf', rf), ('xgb', xgb)], voting='soft')

# Train on SMOTE data
voting_clf.fit(X_train, y_train)

# Predictions
y_pred_voting = voting_clf.predict(X_test)

# Evaluate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print("🔹 Voting Classifier Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_voting):.2f}")
print(f"Precision: {precision_score(y_test, y_pred_voting):.2f}")
print(f"Recall: {recall_score(y_test, y_pred_voting):.2f}")
print(f"F1 Score: {f1_score(y_test, y_pred_voting):.2f}")


🔹 Voting Classifier Performance:
Accuracy: 0.61
Precision: 0.62
Recall: 0.56
F1 Score: 0.59


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define models
log_reg = LogisticRegression(solver='liblinear', random_state=42)  # Works well for imbalanced data
decision_tree = DecisionTreeClassifier(max_depth=5, random_state=42)

# Create Voting Classifier
voting_clf = VotingClassifier(estimators=[('log_reg', log_reg), ('dt', decision_tree)], voting='soft')

# Train on SMOTE data
voting_clf.fit(X_train, y_train)

# Predictions
y_pred_voting = voting_clf.predict(X_test)

# Evaluate
print("🔹 Voting Classifier Performance (Logistic Regression + Decision Tree):")
print(f"Accuracy: {accuracy_score(y_test, y_pred_voting):.2f}")
print(f"Precision: {precision_score(y_test, y_pred_voting):.2f}")
print(f"Recall: {recall_score(y_test, y_pred_voting):.2f}")
print(f"F1 Score: {f1_score(y_test, y_pred_voting):.2f}")


🔹 Voting Classifier Performance (Logistic Regression + Decision Tree):
Accuracy: 0.60
Precision: 0.61
Recall: 0.56
F1 Score: 0.59


hence it is found that RF or RF+XGB can give the best performance